In [2]:
!pip install flask requests_oauthlib requests langchain openai -q


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

import sys  
import os

os.environ['MULTION_CLIENT_ID'] = "<client_id>"
os.environ['MULTION_CLIENT_SECRET'] = "<client_secret>"


sys.path.append('../multion/') 


import multion

multion.login()

 * Serving Flask app 'multion'
 * Debug mode: off


 * Running on https://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [23/Jun/2023 21:19:01] "GET /callback?code=70fc54c0-0180-437f-8506-a1c000237e7e&state=WzS0pNi1OFf3526wYS6T5h0gZxtIaU HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 21:19:01] "GET /favicon.ico HTTP/1.1" 404 -


In [2]:
# Now you can make API calls like this:response = multion.post('https://multion.fly.dev/sessions', {"input": "some input","url": "some url"})
response = multion.new_session({"input": f"Post 'I am excited to start using @MULTION_AI' in twitter","url": "https://www.twitter.com"})
print(response)
tabId = response['response']['data']['tabId']

running post
{'response': {'request_id': '9070f494-bdeb-4b8e-894b-0bb9d776dfbe', 'data': {'url': 'twitter.com', 'screenshot': '', 'tabId': '1821299876', 'message': 'I am typing the desired text into the tweet box and then clicking the Tweet button to post the tweet.\n\n', 'status': 'DONE'}}}


In [3]:
# LANGCHAIN EXAMPLE
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper, SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
import openai


import os

os.environ["LANGCHAIN_TRACING"] = "true"
os.environ['OPENAI_API_KEY'] = "<open_api_key>"



from langchain.tools import StructuredTool
TabId = None

def create_session(action: str, url: str) -> str:
    """Always the first step to run any activities that can be done using browser."""
    global TabId
    response = multion.new_session({"input": action,"url": url})
    
    TabId = response['response']['data']['tabId']
    print(response['response']['data'])
    return  response['response']['data']['message']

def update_session(action:str,url:str) -> str:
    """Updates the existing browser session created using create_session with given action and url, used consequently to handle browser activitites after creating one session of browser"""
    global TabId
    response = multion.update_session(TabId, {"input": action,"url": url})
    print(response['response']['data'])
    return  response['response']['data']['message']

tool1 = StructuredTool.from_function(create_session)
tool2= StructuredTool.from_function(update_session)


llm = OpenAI(temperature=0)

# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type.
agent_executor = initialize_agent(
    [tool1,tool2],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)






In [12]:

agent_executor.run(f"Post 'Message sent by #MULTION_AI' in twitter")




> Entering new  chain...
Action:
```
{
  "action": "create_session",
  "action_input": {
    "action": "open_url",
    "url": "https://twitter.com/"
  }
}
```
running post
Request failed with status code: 404
Response text: {"detail":"WebSocket not found"}


TypeError: 'NoneType' object is not subscriptable